In [ ]:
%matplotlib ipympl
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

In [ ]:
'''ODE vetorial do tipo dy/dt = f(t,y), resolvida de t_0 até t_end, utilizando a condição
inicial y0, do tipo ndarray. É também dado um número de pontos para aplicação do método 
como input da função. A função retorna um array com os valores de tempo t no intervalo 
considerado e os correspondentes valores de y(t)'''

def euler_method(f, t0, y0, t_end, h):

    t_vals = np.arange(t0, t_end + h, h)
    y0 = np.array(y0)  # Ensure y0 is a numpy array
    y_vals = np.zeros((len(t_vals), len(y0)))
    y_vals[0] = y0

    for i in range(len(t_vals) - 1):
        y_vals[i + 1] = y_vals[i] + h * f(t_vals[i], y_vals[i])

    return t_vals, y_vals

In [ ]:
class Coupled_Oscillators:
    def __init__(self, N, m = 1.0, k_springs = 1.0, left_wall_k=0.0, right_wall_k=0.0):

        """Inicializa sistema com N massas e N-1 molas (mais molas opcionais nas paredes).
        Args:
        N (int): number of masses
        m (float ou array): massa de cada oscilador; se um único valor, aplica-se a todas as massas
        k_springs (float ou array): N-1 constantes das molas between; se um único valor, aplica-se a todas as molas
        left_wall_k (float): constante da mola na parede esquerda
        right_wall_k (float): constante da mola na parede direita"""
          
        self.N=N
        self.M = self._build_mass_matrix(m)
        self.M_inv = np.linalg.inv(self.M)
        self.K = self._build_K_matrix(k_springs, left_wall_k, right_wall_k)

    def _build_mass_matrix(self, m):

        if isinstance(m, (int, float)):  
            # se um único valor, aplica-se a todas as massas
            masses = np.full(self.N, m)
        elif isinstance(m, (list, np.ndarray)) and len(m) == self.N:
            masses = np.array(m)
        else:
            raise ValueError("m deve ser um float ou um array de N entradas")
        
        return np.diag(masses)

    def _build_K_matrix(self, k_springs, left_wall_k, right_wall_k):

            N = self.N
            if isinstance(k_springs, (int, float)):
                k_array = np.full(N - 1, k_springs)
            elif isinstance(k_springs, (list, np.ndarray)) and len(k_springs) == N - 1:
                k_array = np.array(k_springs)
            else:
                raise ValueError("k_springs deve ser float ou array de N-1 entradas")

            # Inicializa matriz K
            K = np.zeros((N, N))

            # Preenche matriz com ligações entre massas
            for i in range(N - 1):
                K[i, i] += k_array[i]
                K[i + 1, i + 1] += k_array[i]
                K[i, i + 1] -= k_array[i]
                K[i + 1, i] -= k_array[i]

            # Ligações às paredes
            K[0, 0] += left_wall_k
            K[N - 1, N - 1] += right_wall_k

            return K


    def solve_coupled_system_linear(self, x0=None, v0=None, t_max=50, num_points=10000):
        
        N = self.N
        if x0 is None:
            x0 = np.zeros(N)
        if v0 is None:
            v0 = np.zeros(N)

        y0 = np.concatenate([x0, v0])
        t_span = (0, t_max)
        t_eval = np.linspace(0, t_max, num_points)
        h = t_max / (num_points - 1)

        def system(t, y):
            x = y[:N]
            v = y[N:]
            dxdt = v
            dvdt = -self.M_inv @ self.K @ x
            return np.concatenate([dxdt, dvdt])

        # Solve using solve_ivp
        sol = solve_ivp(system, t_span, y0, t_eval=t_eval, method='RK45')

        # Solve using Euler method
        t_euler, y_euler = euler_method(system, 0, y0, t_max, h)

        # Wrap Euler results similarly
        class EulerSol:
            def __init__(self, t, u):
                self.t = t
                self.u = u.T

        sol_euler = EulerSol(t_euler, y_euler)

        return sol.t, sol.y, sol_euler.t, sol_euler.u


    '''Este método de procurar modos normais é feito para todas as massas iguais: isto
    garante que o produto de M^-1 por K seja uma matriz simétrica. Retorna tuplos (ω,η)
    do vetor e raíz do valor próprios. tolerance é um float positivo usado para evitar 
    erros associados a calcular a raíz de um número negativo e muito próximo de 0 (de-
    rivado de erros intrínsecos ao cálculo dos valores próprios) '''

    def find_normal_modes_and_freqs_for_equal_masses(self, tolerance):

        A = self.M_inv @ self.K
        eigenvalues, eigenvectors = np.linalg.eigh(A)
        modes = []

        for val, vec in zip(eigenvalues, eigenvectors.T):
            if val < -tolerance:
                raise ValueError(f"Valor próprio negativo encontrado: {val}")
            elif val < 0:
                omega = 0
            else: 
                omega = np.sqrt(val)

            vec = vec / np.linalg.norm(vec)
            modes.append((omega, vec))
        
        return modes

### Exercício 2

In [ ]:
# Função para desenhar os gráficos de cada massa (posições)

def plot_displacements(t, rk_positions, euler_positions, title_prefix):
    N = rk_positions.shape[0]
    _, axes = plt.subplots(N, 1, figsize=(10, 2 * N), sharex=True)
    for i in range(N):
        ax = axes[i] if N > 1 else axes
        ax.plot(t, rk_positions[i], label='Runge-Kutta')
        ax.plot(t, euler_positions[i], '--', label='Euler')
        ax.set_ylabel(f'Massa {i}')
        ax.grid(True)
        if i == 0: ax.set_title(title_prefix)
    axes[-1].set_xlabel('Tempo (s)')
    plt.legend()
    plt.tight_layout()
    plt.show()


# Cálculo das energias cinética, potencial e total e respetivo plot

def plot_energies(t, rk_solution, euler_solution, m_matrix, k_matrix, title):
    
    max_t_index = len(t)
    T_rk, T_euler, V_rk, V_euler = [], [], [], []
    N_masses = len(rk_solution)//2 #pois a solução tem guardadas as posições e as velocidades

    for i in range(max_t_index):
        
        euler_x_vector = np.array([euler_solution[mass][i] for mass in range(N_masses)])
        euler_v_vector = np.array([euler_solution[mass+N_masses][i] for mass in range(N_masses)])
        rk_x_vector = np.array([rk_solution[mass][i] for mass in range(N_masses)])
        rk_v_vector = np.array([rk_solution[mass+N_masses][i] for mass in range(N_masses)])

        RK_T = 0.5 * rk_v_vector.T @ m_matrix @ rk_v_vector
        RK_V = 0.5 * rk_x_vector.T @ k_matrix @ rk_x_vector
        EU_T = 0.5 * euler_v_vector.T @ m_matrix @ euler_v_vector
        EU_V = 0.5 * euler_x_vector.T @ k_matrix @ euler_x_vector

        T_rk.append(RK_T) 
        V_rk.append(RK_V)
        T_euler.append(EU_T)
        V_euler.append(EU_V)

    T_rk = np.array(T_rk)
    V_rk = np.array(V_rk)
    T_euler = np.array(T_euler)
    V_euler = np.array(V_euler)

    E_rk = T_rk + V_rk
    E_euler = T_euler + V_euler

    plt.figure(figsize=(10, 6))
    plt.plot(t, T_rk, label='T (RK)', color='tab:blue')
    plt.plot(t, V_rk, label='V (RK)', color='tab:orange')
    plt.plot(t, E_rk, label='E (RK)', color='tab:green')

    plt.plot(t, T_euler, '--', label='T (Euler)', color='tab:blue')
    plt.plot(t, V_euler, '--', label='V (Euler)', color='tab:orange')
    plt.plot(t, E_euler, '--', label='E (Euler)', color='tab:green')

    plt.title(title)
    plt.xlabel('Tempo (s)')
    plt.ylabel('Energia (J)')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()



# Parâmetros

N = 5              # Número de massas
m, k = 1.0, 1.0    # Massa [kg] e constante elástica [N/m]
delta_x = 0.01     # Deslocamento [m] a utilizar nas simulações
t_max = 30

# Simulação 2a: Apenas a massa central é deslocada
osc_2a = Coupled_Oscillators(N, m, k, k, k)
x0_2a = [delta_x if i == N // 2 else 0 for i in range(N)]
t, rk_2a, _, euler_2a = osc_2a.solve_coupled_system_linear(x0=x0_2a, t_max=t_max)
plot_displacements(t, rk_2a[:N], euler_2a[:N], "2a) Deslocamento apenas da massa central")
plot_energies(t, rk_2a, euler_2a, osc_2a.M, osc_2a.K, "2a) Massa central deslocada")

# Simulação 2b: Todas as massas deslocadas igualmente no início
osc_2b = Coupled_Oscillators(N, m, k, k, k)
x0_2b = np.full(N, delta_x)
t, rk_2b, _, euler_2b = osc_2b.solve_coupled_system_linear(x0=x0_2b, t_max=10)
plot_displacements(t, rk_2b[:N], euler_2b[:N], "2b) Todas as massas deslocadas igualmente")
plot_energies(t, rk_2b, euler_2b, osc_2b.M, osc_2b.K, "2b) Todas as massas deslocadas igualmente")


### Exercício 3

In [ ]:
# Mostra o vetor de cada modo normal
def plot_normal_modes(modes):
    # Para cada modo normal (ω, vetor η)7

    for j, (omega, eta) in enumerate(modes):
        plt.figure(figsize=(8, 3))
        plt.plot(eta, 'o-', label=f"Modo {j+1} (ω = {omega:.2f} rad/s)")
        plt.title(f"Modo Normal {j+1}")
        plt.xlabel("Índice da Massa")
        plt.xticks(ticks=np.arange(len(eta)))
        plt.ylabel("Amplitude Relativa")
        plt.ylim(-1, 1)
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.show()

# Combina os modos normais com as condições iniciais e devolve as funções x_i(t)
def normal_mode_combination(modes, M, x0, v0):
    Mx0 = M @ x0
    Mv0 = M @ v0

    coeffs = []
    for omega, eta in modes:
        a = eta @ Mx0
        b = omega * (eta @ Mv0)
        coeffs.append((omega, a, b, eta))

    def x_i_func(i):
        # Retorna função x_i(t) para a massa i
        def x_i(t):
            terms = [(a*np.cos(ω*t) + b*np.sin(ω*t)) * η[i] for ω, a, b, η in coeffs]
            return np.sum(terms, axis=0)
        return x_i

    N = len(x0)
    return [x_i_func(i) for i in range(N)]

# Mostra o gráfico de x_i(t) para cada massa i
def plot_modes_displacements(N, t_array, x_funcs, title):
    fig, axes = plt.subplots(N, 1, figsize=(10, 2*N), sharex=True)

    for i in range(N):
        xi_t = [x_funcs[i](t) for t in t_array]
        ax = axes[i] if N > 1 else axes
        ax.plot(t_array, xi_t)
        ax.set_ylabel(f'Massa {i}')
        ax.grid(True)
        if i == 0: ax.set_title(title)

    axes[-1].set_xlabel('Tempo (s)')
    plt.tight_layout()
    plt.show()


# Simulações pedidas na pergunta 3

N_masses = 5
osc = Coupled_Oscillators(N_masses)
modes = osc.find_normal_modes_and_freqs_for_equal_masses(1e-8)
plot_normal_modes(modes)

# Condições iniciais
x0 = np.zeros(N_masses)
x0[0] = 0.01
v0 = np.zeros(N_masses)
t = np.linspace(0, t_max, 10000)

# Calcula x_i(t) e plota
x_funcs = normal_mode_combination(modes, osc.M, x0, v0)
plot_modes_displacements(N_masses, t, x_funcs, "Combinação de Modos Normais")

### Exercício 4  - Trabalho adicional

**Parte I - Animação** 

Nesta parte fazemos uma animação dos modos normais, utilizando o módulo animation do matplotlib. Para tal foi necessário utilizar o pacote ipympl, instalado com o pip. A animação funciona fazendo um gráfico estático e redesenhando-o sucessivamente com os dados atualizados, neste caso das posições das massas nos modos normais.

In [ ]:
import matplotlib.animation as animation

#Declarar imagem geral
fig, axes = plt.subplots(5, 1, figsize=(10, 20), sharex=True)
axes[0].set_title("Animações dos modos normais")

#Variáveis para resolver o sistema
t_max = 50
n = 1000
v0 = np.zeros(5)

#Nesta lista serão armazenados os desenhos de cada modo normal
line = []

for i in range(len(modes)):
    ax = axes[i]
    ax.set_xlabel('Índice da massa')
    ax.set_ylabel('Posição (m)')
    ax.grid(True)

    #Definição do modo normal i
    omega = modes[i][0]
    eta = modes[i][1]

    #Colocar modo normal i na lista de modos normais
    line.append(ax.plot(eta, 'o-', label=f"Modo {i} (ω = {omega:.2f} rad/s)")[0])

def update(frame):
    #Para cada modo normal, atualizar a posição em cada frame para a solução após esse número de passos
    for i in range(len(modes)):
        eta = modes[i][1] #Redeclarar o modo normal para usar como condição inicial
        _, pos, _, _ = osc.solve_coupled_system_linear(eta, v0, num_points=n) #Resolver o sistema

        #Atualização do gráfico
        line[i].set_ydata(pos[:5, frame-1])

    return line

#Declarar a animação
anim = animation.FuncAnimation(fig=fig, func=update, frames=n, interval=50)

plt.show()

**Parte II - Oscilador Forçado e com atrito** 

Nesta parte, é introduzida uma extensão do sistema de osciladores acoplados para incluir os efeitos de atrito (amortecimento viscoso) e uma força externa harmónica aplicada a uma das massas. Para tal, foi criada a subclasse ForcedDampedOscillator, que herda da classe base de osciladores acoplados.

A força externa tem amplitude F0 e frequência angular w, sendo aplicada a uma massa específica do sistema, indicada pelo índice forced_mass_index. O termo de atrito é proporcional à velocidade de cada massa e controlado pelo coeficiente c.

O sistema é resolvido numericamente através de dois métodos:

Runge-Kutta (solve_ivp), para maior precisão;

Euler, para comparação com um método mais simples.

In [ ]:
# Subclasse para incluir atrito e força externa no sistema
class ForcedDampedOscillator(Coupled_Oscillators):
    def __init__(self, N, m=1.0, k_springs=1.0, c=0.5, F0=1.0, omega_ext=1.0, forced_mass_index=0,
                 left_wall_k=0.0, right_wall_k=0.0):
        super().__init__(N, m, k_springs, left_wall_k, right_wall_k)
        self.c = c
        self.F0 = F0
        self.omega_ext = omega_ext
        self.forced_mass_index = forced_mass_index

    def solve_forced_system(self, x0=None, v0=None, t_max=50, num_points=10000):
        N = self.N
        if x0 is None:
            x0 = np.zeros(N)
        if v0 is None:
            v0 = np.zeros(N)

        y0 = np.concatenate([x0, v0])
        t_span = (0, t_max)
        t_eval = np.linspace(*t_span, num_points)
        h = t_max / (num_points - 1)

        def system(t, y):
            x = y[:N]
            v = y[N:]
            dxdt = v
            damping = -self.c * v
            restoring = -self.K @ x
            forcing = np.zeros(N)
            forcing[self.forced_mass_index] = self.F0 * np.cos(self.omega_ext * t)
            dvdt = self.M_inv @ (restoring + damping + forcing)
            return np.concatenate([dxdt, dvdt])

        # Resolver com Runge-Kutta
        sol = solve_ivp(system, t_span, y0, t_eval=t_eval)

        # Euler
        t_euler, y_euler = euler_method(system, 0, y0, t_max, h)

        class EulerSol:
            def __init__(self, t, u):
                self.t = t
                self.u = u.T

        sol_euler = EulerSol(t_euler, y_euler)

        return sol.t, sol.y, sol_euler.t, sol_euler.u
    
    # Simulação com atrito e força externa numa das massas
N = 5
m, k = 1.0, 1.0
delta_x = 0.01
t_max = 50

# Oscilador com atrito e força aplicada à massa do meio
osc_forced = ForcedDampedOscillator(N, m, k, c=0, F0=10, omega_ext=1.0, forced_mass_index=N//2)
x0 = np.zeros(N)
v0 = np.zeros(N)
v0[N // 2] = 1.0  # a massa central começa com velocidade 1.0

t, rk_fd, _, euler_fd = osc_forced.solve_forced_system(x0, v0, t_max=t_max)

plot_displacements(t, rk_fd[:N], euler_fd[:N], "Oscilador Forçado com Atrito na Massa Central")
plot_energies(t, rk_fd, euler_fd, osc_forced.M, osc_forced.K, "Energia - Oscilador Forçado com Atrito")



**Parte III - Simulação com Termo Cúbico Não Linear** 

Nesta parte, é analisado o comportamento de um sistema de osciladores acoplados que apresenta uma não linearidade cúbica na força de retorno. Para isso, foi criada a subclasse CubicNonlinearOscillator, que estende a classe base de osciladores introduzindo um termo proporcional ao cubo do deslocamento (𝛼𝑥^3) na equação do movimento.

Este tipo de não linearidade representa sistemas físicos onde a força restauradora não segue exatamente a Lei de Hooke (linear), mas cresce de forma mais intensa com o deslocamento — fenômeno que pode ocorrer, por exemplo, em molas reais sob grandes deformações.

A evolução temporal do sistema é obtida numericamente através de dois métodos:

Runge-Kutta (solve_ivp), para maior precisão na integração das equações diferenciais não lineares;

Euler, para comparação com um método mais simples e mais suscetível a instabilidades numéricas.

In [ ]:
# Subclasse para incluir um termo cúbico na força de retorno
class CubicNonlinearOscillator(Coupled_Oscillators):
    def __init__(self, N, m=1.0, k_springs=1.0, alpha=1.0, left_wall_k=0.0, right_wall_k=0.0):
        super().__init__(N, m, k_springs, left_wall_k, right_wall_k)
        self.alpha = alpha  # Coeficiente do termo cúbico

    def solve_nonlinear_cubic(self, x0=None, v0=None, t_max=50, num_points=10000):
        N = self.N
        if x0 is None:
            x0 = np.zeros(N)
        if v0 is None:
            v0 = np.zeros(N)

        y0 = np.concatenate([x0, v0])
        t_span = (0, t_max)
        t_eval = np.linspace(*t_span, num_points)
        h = t_max / (num_points - 1)

        def system(t, y):
            x = y[:N]
            v = y[N:]
            dxdt = v
            restoring = self.K @ x
            cubic_force = self.alpha * x**3  # termo cúbico aplicado componente a componente
            dvdt = -self.M_inv @ (restoring + cubic_force)
            return np.concatenate([dxdt, dvdt])

        # Resolver com Runge-Kutta
        sol = solve_ivp(system, t_span, y0, t_eval=t_eval)

        # Euler
        t_euler, y_euler = euler_method(system, 0, y0, t_max, h)

        class EulerSol:
            def __init__(self, t, u):
                self.t = t
                self.u = u.T

        sol_euler = EulerSol(t_euler, y_euler)

        return sol.t, sol.y, sol_euler.t, sol_euler.u

# Simulação com termo cúbico não linear
N = 5
m, k = 1.0, 1.0
alpha = 100.0  # intensidade do termo cúbico
delta_x = 0.01
t_max = 50

# Criar sistema não linear
osc_nonlinear = CubicNonlinearOscillator(N, m, k, alpha=alpha)
x0 = [delta_x if i == N // 2 else 0 for i in range(N)]  # deslocamento inicial só na massa central
v0 = np.zeros(N)

t, rk_nl, _, euler_nl = osc_nonlinear.solve_nonlinear_cubic(x0=x0, v0=v0, t_max=t_max)

plot_displacements(t, rk_nl[:N], euler_nl[:N], "Oscilador com Termo Cúbico Não Linear")
plot_energies(t, rk_nl, euler_nl, osc_nonlinear.M, osc_nonlinear.K, "Energia - Sistema Não Linear Cúbico")


**Parte IV - Análise Espectral com Transformada de Fourier (FFT)** 

Nesta parte, é realizada uma análise espectral da resposta temporal de uma das massas do sistema linear de osciladores acoplados, com o objetivo de identificar as frequências naturais de oscilação. Para isso, é utilizada a Transformada de Fourier Rápida (FFT), através da função fft da biblioteca scipy.

In [ ]:
from scipy.fft import fft, fftfreq
import matplotlib.pyplot as plt

def plot_fft(signal, t, title="FFT da Massa", sampling=None):
    N = len(t)
    T = t[1] - t[0] if sampling is None else 1 / sampling
    yf = fft(signal)
    xf = fftfreq(N, T)[:N // 2]
    magnitude = 2.0 / N * np.abs(yf[:N // 2])

    plt.figure(figsize=(10, 4))
    plt.plot(xf, magnitude)
    plt.title(title)
    plt.xlabel("Frequência (Hz)")
    plt.ylabel("Amplitude")
    plt.grid()
    plt.tight_layout()
    plt.show()

# Simulação linear com excitação inicial numa massa
N = 5
m, k = 1.0, 1.0
delta_x = 1
t_max = 100

osc = Coupled_Oscillators(N, m, k)
x0 = np.zeros(N)
v0 = np.zeros(N)
v0[N // 2] = 0.03  # a massa central começa com velocidade 1.0
x0[N // 2] = delta_x  # Excitação na massa do meio

t, y_rk, _, _ = osc.solve_coupled_system_linear(x0, v0, t_max=t_max)

# FFT da massa central
massa_index = 3
signal = y_rk[massa_index]

plot_fft(signal, t, title=f"FFT da Massa {massa_index}")
